In [1]:
import pandas as pd
import pprint
import json 

from openai import AsyncOpenAI
import asyncio
from tqdm.asyncio import tqdm
import tiktoken

import os
from dotenv import load_dotenv

load_dotenv()
os.getenv('OPENAI_API_KEY')

client = AsyncOpenAI()
model = "gpt-4o-mini"

In [2]:
def read_parquet(uploaded_file):
    return pd.read_parquet(uploaded_file)

df = read_parquet("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024.parquet")
df.reset_index(inplace=True)
df_claims = df.explode('claims', ignore_index=True)
df_claims = pd.json_normalize(df_claims['claims'])
df_claims = df_claims[(df_claims['disinformation_score']=='high') & (df_claims['pro_anti']!='pro-écologie')]
df_claims

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
67,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
87,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative
251,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative
259,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative
313,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative
...,...,...,...,...,...,...,...,...,...
73775,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative
74166,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,narrative
74219,"Cette affirmation semble contestable, car de n...",Le risque de chutes de séracs n'est pas partic...,"Le chercheur indique que, pour l'instant, le r...",N/A,consensus,high,anti-écologie,"Pour l'instant, le risque de chutes de séracs ...",narrative
74850,Cette information est manifestement fausse et ...,Une astuce sur les réseaux sociaux prétend que...,Un segment aborde une nouvelle tendance sur le...,N/A,narrative,high,anti-écologie,Cette astuce elle est fausse et voilà pourquoi.,narrative


In [3]:
system_prompt = f"""
Tu es expert en désinformation sur les sujets environnementaux, expert en science climatique et sachant tout sur le GIEC. 
Je vais te donner des claims sur des thématiques environnementales extraits de retranscriptions d'un flux TV ou Radio à factchecker. 
Pour chaque claim, donne une catégorisation de cette allégation suivant la classification qui suit.
Renvoie le résultat en json sans autre phrase d'introduction ou de conclusion avec à chaque fois le champ suivant : "cards_label_pred"

<catégories prédéfinies>
- 0_accepted: Lorsque l'ensemble de l'extrait est neutre ou reflète le consensus scientifique sur les questions climatiques, sans contenir de désinformation. Cette catégorie est aussi adaptée si l'extrait n'aborde pas le sujet du climat.
- 1_its_not_happening: Le réchauffement climatique n'a pas lieu. Cette catégorie soutient que le changement climatique est inexistant ou largement exagéré. Elle suggère que des facteurs naturels — tels que le froid, la préservation de la glace et le refroidissement des océans — prouvent que le réchauffement n'est pas en cours. De plus, elle implique que les prévisions de changements climatiques sévères, y compris la montée du niveau de la mer et les phénomènes météorologiques extrêmes, sont exagérées ou inventées. Certaines affirmations soutiennent que la terminologie climatique a été manipulée (par exemple, en passant de "réchauffement climatique" à "changement climatique") pour alarmer le public sans raison.
- 2_its_not_us: Les humains ne sont pas à l'origine du réchauffement climatique. Cette catégorie avance que les activités humaines, en particulier les émissions de gaz à effet de serre, ne sont pas responsables du réchauffement observé. Elle soutient que les cycles naturels, l'activité solaire ou d'autres facteurs non humains sont à l'origine des changements climatiques. Les affirmations de cette catégorie minimisent souvent le rôle du CO2, ou soulignent que les changements climatiques passés se sont produits sans intervention humaine, suggérant que le réchauffement actuel est également naturel.
- 3_its_not_bad: Les impacts climatiques ne sont pas mauvais. Cette catégorie reconnaît qu'un certain réchauffement pourrait se produire, mais affirme qu'il pose peu de risques. Les arguments de cette catégorie suggèrent que la sensibilité du climat au CO2 est faible, ce qui limiterait le réchauffement. Les partisans prétendent que l'environnement, les espèces et la santé humaine seront largement épargnés, car les plantes bénéficient de l'augmentation du CO2, et que de nombreuses espèces, comme les ours polaires, peuvent s'adapter. Dans l'ensemble, elle suggère que les impacts climatiques seront mineurs et gérables.
- 4_solutions_wont_work: Les solutions climatiques ne fonctionneront pas. Cette catégorie s'oppose aux politiques climatiques proposées, en affirmant qu'elles seront nuisibles économiquement ou socialement, inefficaces ou irréalistes. Elle soutient que les politiques de réduction des émissions augmenteront les coûts, affaibliront la sécurité nationale et restreindront les libertés individuelles sans bénéfices climatiques significatifs. D'autres affirmations proposent que les solutions "vertes" sont peu fiables, que l'énergie propre est non éprouvée, ou que les combustibles fossiles sont abondants et essentiels pour l'avenir. Essentiellement, elle soutient que les politiques climatiques sont trop coûteuses, difficiles ou inefficaces pour être poursuivies.
- 5_science_is_unreliable: Les mouvements pour le climat et la science du climat ne sont pas fiables. Cette catégorie attaque la crédibilité de la science climatique et du mouvement pour le climat, affirmant qu'ils sont non fiables, biaisés ou idéologiquement orientés. Elle remet en question le consensus scientifique sur le changement climatique, suggérant que les données, modèles et méthodes sont défectueux. De plus, elle présente le mouvement climatique comme alarmiste, biaisé ou faisant partie d'une conspiration plus large, motivée par des intentions politiques ou religieuses plutôt qu'une véritable préoccupation. Ce cadre cherche à miner la confiance dans la science climatique et dans ceux qui plaident pour l'action.
</catégories prédéfinies>
    """

encoding = tiktoken.encoding_for_model(model)
prompt_tokens = len(encoding.encode(system_prompt))

async def classify_claim(
        text: str, system_prompt: str = system_prompt, prompt_tokens: str = prompt_tokens, tpm_limit: int = 2e6
    ):
    text_tokens = len(encoding.encode(text))
    total_tokens = prompt_tokens + text_tokens
    wait_time_s = 60 * total_tokens / tpm_limit
    await asyncio.sleep(wait_time_s)
    
    response = await client.beta.chat.completions.parse(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": text},
        ],
        temperature=0,
    )
    return response.choices[0].message.content

async def run_classification(texts: list[str]):
    semaphore = asyncio.Semaphore(50)
    
    async def bounded_classify_claim(text):
        async with semaphore:
            return await classify_claim(text)
    
    classification = await tqdm.gather(*[bounded_classify_claim(text) for text in texts])
    return classification

In [11]:
classification = await run_classification(df_claims["claim"])

100%|██████████| 1454/1454 [00:31<00:00, 46.60it/s]


In [14]:
len(classification)

1454

In [15]:
len(df_claims)

1454

In [16]:
import json
import re


# Liste pour stocker les valeurs de "cards_label_pred"
cards_label_preds = []

# Parcourir chaque élément de data
for item in classification:
    # Retirer les balises ```json et ``` si présentes
    cleaned_item = re.sub(r'```json\n|```', '', item)

    # Charger en tant que dictionnaire JSON
    try:
        parsed_item = json.loads(cleaned_item)
        # Extraire la valeur de "cards_label_pred"
        cards_label_preds.append(parsed_item["cards_label_pred"])
    except json.JSONDecodeError:
        print(f"Erreur de parsing pour l'élément : {cleaned_item}")

# Afficher le résultat
cards_label_preds

['5_science_is_unreliable',
 '5_science_is_unreliable',
 '0_accepted',
 '5_science_is_unreliable',
 '5_science_is_unreliable',
 '5_science_is_unreliable',
 '3_its_not_bad',
 '2_its_not_us',
 '5_science_is_unreliable',
 '4_solutions_wont_work',
 '5_science_is_unreliable',
 '0_accepted',
 '1_its_not_happening',
 '0_accepted',
 '0_accepted',
 '0_accepted',
 '0_accepted',
 '3_its_not_bad',
 '0_accepted',
 '0_accepted',
 '0_accepted',
 '1_its_not_happening',
 '2_its_not_us',
 '5_science_is_unreliable',
 '5_science_is_unreliable',
 '0_accepted',
 '0_accepted',
 '4_solutions_wont_work',
 '0_accepted',
 '1_its_not_happening',
 '4_solutions_wont_work',
 '4_solutions_wont_work',
 '0_accepted',
 '0_accepted',
 '3_its_not_bad',
 '0_accepted',
 '0_accepted',
 '1_its_not_happening',
 '4_solutions_wont_work',
 '5_science_is_unreliable',
 '0_accepted',
 '0_accepted',
 '4_solutions_wont_work',
 '5_science_is_unreliable',
 '4_solutions_wont_work',
 '4_solutions_wont_work',
 '4_solutions_wont_work',
 '0_

In [17]:
len(cards_label_preds)

1454

In [19]:
df_claims["cards_label_pred"] = cards_label_preds

In [20]:
df_claims

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,cards_label_pred
67,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
87,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
251,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative,0_accepted
259,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative,5_science_is_unreliable
313,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative,5_science_is_unreliable
...,...,...,...,...,...,...,...,...,...,...
73775,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative,0_accepted
74166,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,narrative,3_its_not_bad
74219,"Cette affirmation semble contestable, car de n...",Le risque de chutes de séracs n'est pas partic...,"Le chercheur indique que, pour l'instant, le r...",N/A,consensus,high,anti-écologie,"Pour l'instant, le risque de chutes de séracs ...",narrative,1_its_not_happening
74850,Cette information est manifestement fausse et ...,Une astuce sur les réseaux sociaux prétend que...,Un segment aborde une nouvelle tendance sur le...,N/A,narrative,high,anti-écologie,Cette astuce elle est fausse et voilà pourquoi.,narrative,0_accepted


In [31]:
df_exploded["cards_label_pred"] = ''
df_exploded.loc[df_claims.index, "cards_label_pred"] = df_claims["cards_label_pred"]

In [33]:
df_exploded[df_exploded["cards_label_pred"]!='']

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,cards_label_pred
67,f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c...,2023-09-04 05:56:00,série qu'on peut redire par fonction juré déjà...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169379984302...",2,559,{'analysis': 'Cette allégation semble associée...,5_science_is_unreliable
87,e57e39657004072e5f998d19e6a8a952ee8f35923b9072...,2023-09-04 07:50:00,salle d'attente il y a toutes les classes rage...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_causes"", ""changement_c...","[{""keyword"": ""giec"", ""timestamp"": 169380664205...",2,554,{'analysis': 'Cette allégation semble utiliser...,5_science_is_unreliable
251,5b7207073227562ad5fd7b252e290d0a2079f3cbbb2e5c...,2023-09-06 07:22:00,en france on peut quand même s'appuyer sur ce ...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences_indirecte...","[{""keyword"": ""incendies"", ""timestamp"": 1693977...",2,594,{'analysis': 'Cette allégation semble hautemen...,0_accepted
259,53b6b045672a2327c58c72a38a3ddb603587ad43421f8c...,2023-09-06 08:38:00,je suis même allé plus loin je vais essayer de...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_consequences"", ""ressou...","[{""keyword"": ""\u00e9coresponsable"", ""timestamp...",2,614,{'analysis': 'Cette allégation frôle la théori...,5_science_is_unreliable
313,6c3bb9299d31332c644b54832399976117d28776c9e22c...,2023-09-07 08:56:00,pour faire réussir nos enfants dans quel monde...,europe1,True,Information - Magazine,L'heure des pros,"[""changement_climatique_constat""]","[{""keyword"": ""climatique"", ""timestamp"": 169406...",1,563,{'analysis': 'Si le changement climatique est ...,5_science_is_unreliable
...,...,...,...,...,...,...,...,...,...,...,...,...,...
73775,f875741067fda8011d4aa9ad4c543e52227e31b525fed1...,2024-07-29 13:14:00,sur place voilà ce qu'ils restent par exemple ...,tf1,False,Information - Journal,JT 13h,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""feux"", ""timestamp"": 172225164506...",1,446,{'analysis': 'Cette allégation est sérieuse et...,0_accepted
74166,cdfed233896d49971897df448465dd43b79ae871b33adb...,2024-08-05 09:16:00,les insectes non on pense que c'est la chaleur...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""g\u00e9othermie"", ""timestamp"": 1...",2,479,{'analysis': 'Ce claim fait l'impasse sur le f...,3_its_not_bad
74219,9441e27a4c144b4ceef27cb8bd44ae5b7ae0fd545fde2b...,2024-08-06 09:04:00,chute de séracs sont communs commune à toutes ...,tf1,False,Information - Magazine,Bonjour ! La Matinale,"[""changement_climatique_constat_indirectes"", ""...","[{""keyword"": ""climatique"", ""timestamp"": 172292...",1,400,{'analysis': 'Cette affirmation semble contest...,1_its_not_happening
74850,6f48350bc56ebb262e90f9950e95de345ce69e3fb0053b...,2024-08-17 20:24:00,le vaucluse ses problèmes d'étiquetage représe...,tf1,False,Information - Journal,JT 20h + météo,"[""biodiversite_concepts_generaux"", ""biodiversi...","[{""keyword"": ""agriculteur"", ""timestamp"": 17239...",1,494,{'analysis': 'Cette information est manifestem...,0_accepted


In [34]:
df_exploded

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,cards_label_pred
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,{'analysis': 'Cette allégation nécessite une v...,
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'L'allégation semble être fondée ...,
2,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cet énoncé reflète un changement...,
3,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cette allégation semble provoque...,
4,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cette affirmation sur la croissa...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75308,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,{'analysis': 'Cette affirmation peut refléter ...,
75309,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,{'analysis': 'Il est bien documenté que les in...,
75310,ec42467d13f38ae9d625599a61d9acc71b4507c370525f...,2024-08-31 20:34:00,viticoles françaises la saison des vendanges a...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""for\u00eat"", ""timestamp"": 172512...",3,391,{'analysis': 'L'allégation semble fondée car d...,
75311,ec42467d13f38ae9d625599a61d9acc71b4507c370525f...,2024-08-31 20:34:00,viticoles françaises la saison des vendanges a...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""for\u00eat"", ""timestamp"": 172512...",3,391,{'analysis': 'Bien que cela puisse être vrai d...,


In [37]:
def add_key(row):
    try:
        return {**row['claims'], 'cards_label_pred': row['cards_label_pred']}
    except:
        return row['claims']


df_exploded['claims'] = df_exploded.apply(lambda row: add_key(row), axis=1)

In [38]:
df_exploded

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims,cards_label_pred
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,{'analysis': 'Cette allégation nécessite une v...,
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'L'allégation semble être fondée ...,
2,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cet énoncé reflète un changement...,
3,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cette allégation semble provoque...,
4,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,{'analysis': 'Cette affirmation sur la croissa...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
75308,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,{'analysis': 'Cette affirmation peut refléter ...,
75309,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,{'analysis': 'Il est bien documenté que les in...,
75310,ec42467d13f38ae9d625599a61d9acc71b4507c370525f...,2024-08-31 20:34:00,viticoles françaises la saison des vendanges a...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""for\u00eat"", ""timestamp"": 172512...",3,391,{'analysis': 'L'allégation semble fondée car d...,
75311,ec42467d13f38ae9d625599a61d9acc71b4507c370525f...,2024-08-31 20:34:00,viticoles françaises la saison des vendanges a...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""for\u00eat"", ""timestamp"": 172512...",3,391,{'analysis': 'Bien que cela puisse être vrai d...,


In [41]:
# Supprimer la colonne 'label'
df_exploded = df_exploded.drop(columns=['cards_label_pred'])

In [47]:
df_exploded.duplicated('id')

0        False
1        False
2         True
3         True
4         True
         ...  
75308     True
75309     True
75310    False
75311     True
75312     True
Length: 75313, dtype: bool

In [48]:
df_grouped = df_exploded.groupby('id', as_index=False).agg({'claims': list})

In [51]:
df_grouped.iloc[0]['claims']

[nan]

In [57]:
df_grouped.loc[65, 'claims']

[{'analysis': "L'allégation selon laquelle les pouvoirs occultes peuvent influencer la santé mentale est problématique, car elle repose sur des idées non scientifiques et non prouvées. Cela peut encourager des pratiques basées sur des croyances sans fondement plutôt que sur des traitements médicaux validés.",
  'claim': 'Le pouvoir occultes ont un impact sur la santé mentale des individus.',
  'context': 'Un intervenant discute des effets supposés de pouvoirs occultes sur les personnes en souffrance, en mentionnant des histoires personnelles et des approches psychologiques liées à ces croyances.',
  'contradiction': 'N/A',
  'disinformation_category': 'narrative',
  'disinformation_score': 'high',
  'pro_anti': 'anti-écologie',
  'quote': "il est en prise avec les démons de sa vie c'est-à-dire dix c'est peu de diverses aides des addictions l'alcoolisme et séparations",
  'speaker': 'consensus',
  'cards_label_pred': '0_accepted'}]

In [62]:
import math
math.isnan(df_grouped.iloc[0].claims[0])

True

In [67]:
def isnan_(row):
    try:
        return math.isnan(row.claims[0])
    except:
        return False

In [72]:
df_grouped.apply(lambda row: isnan_(row), axis=1)

0         True
1        False
2        False
3        False
4        False
         ...  
39126    False
39127     True
39128     True
39129    False
39130    False
Length: 39131, dtype: bool

In [69]:
df_grouped

,id,claims
0,0005675934b519565480256ef9358c0e980961740b8f05...,[nan]
1,00057a237e27df45620beb329d9203ddba9749a67b3b94...,[{'analysis': 'Cette allégation semble plausib...
2,000622d428983d340bf4dbd12f7ca1ccd48bd25731f1e2...,[{'analysis': 'Bien que cette allégation décri...
3,0006474c4f6b7088e9ce1b89b902d454be7fd446085c48...,[{'analysis': 'L'allégation est correcte en ce...
4,0007c029cf9cf2c43b99a870fea302f2304539eb0253f4...,[{'analysis': 'Cette affirmation est fondée su...
...,...,...
39126,fff76f2bd5c0c56fbb6d2c06a15bdca75dfde53a4df64f...,[{'analysis': 'Cette allégation souligne l'imp...
39127,fffb24800a34c4a01ef2a38077bbf0fb6ba3604879d120...,[nan]
39128,fffc1b8333086b9044e997b33fb890682c4678bf53aba6...,[nan]
39129,fffc87c2d670e5368f1a5e0f43a430c4fdf99bf5042f1c...,[{'analysis': 'Cette allégation est correcte c...


In [78]:
df.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims'] = df_grouped.loc[~ df_grouped.apply(lambda row: isnan_(row), axis=1), 'claims']

In [79]:
df

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,[{'analysis': 'Cette allégation nécessite une ...
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,[{'analysis': 'Cette allégation semble plausib...
2,b6d54aefb250671e7754a688411ce9e68badcad88a665b...,2023-09-01 07:38:00,mais titeuf ne vieillit pas le monde change ti...,europe1,True,Information - Magazine,Europe 1 Matin,"[""changement_climatique_constat""]","[{""keyword"": ""bilan carbone"", ""timestamp"": 169...",1,606,[{'analysis': 'Bien que cette allégation décri...
3,23c2d3b292d9ab0fb0d0b2b8c34f3b88708c79ffab2d66...,2023-09-01 08:44:00,dû travailler très vite le journal arrive à no...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""ressources...","[{""keyword"": ""eaux"", ""timestamp"": 169355074908...",3,515,[{'analysis': 'L'allégation est correcte en ce...
4,71df0ce2b34afa23391d8e31d35ccd213ae2a881b7ce41...,2023-09-01 08:46:00,pas avoir lieu ni même européens existait ni l...,europe1,True,Information - Magazine,Europe 1 Matin,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""\u00e9cologiste"", ""timestamp"": 1...",2,536,[{'analysis': 'Cette affirmation est fondée su...
...,...,...,...,...,...,...,...,...,...,...,...,...
39126,7e43ce79a3c9304277dde3a06b433d7cf08a9b40ee8904...,2024-08-31 13:08:00,lui aussi par les moustiques touche une trenta...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""biodiversite_co...","[{""keyword"": ""renaturer"", ""timestamp"": 1725102...",4,474,[{'analysis': 'Cette allégation souligne l'imp...
39127,4b5a990a668084eb50da9029b16f7f76ba44ffc8941285...,2024-08-31 13:10:00,des griffes arracher la solution se trouve peu...,tf1,False,Information - Journal,JT 13h,"[""biodiversite_consequences"", ""attenuation_cli...","[{""keyword"": ""agriculture"", ""timestamp"": 17251...",3,471,[{'analysis': 'L'allégation qu'une plante a de...
39128,d3ef252aaac62bdcd506e162d5454dfa346f9c75dbc74e...,2024-08-31 19:58:00,albret <unk> madame monsieur bonsoir bienvenue...,tf1,False,Information - Journal,JT 20h + météo,"[""changement_climatique_consequences""]","[{""keyword"": ""al\u00e9as climatiques"", ""timest...",1,419,[{'analysis': 'L'allégation sur la qualité de ...
39129,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,[{'analysis': 'Cette allégation est correcte c...


In [80]:
df.to_csv("/Users/anne/Desktop/4_channels_predictions_09_2023_09_2024_with_cards.csv")

In [21]:
df_exploded = df.explode('claims', ignore_index=True)
df_normalized = pd.json_normalize(df_exploded['claims'])
df_result = df_exploded.drop(columns=['claims']).join(df_normalized)
df_result

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker
0,1dcd4b454f8bac42440259ce26a1a2192051186bb5728b...,2023-09-01 06:26:00,<unk> <unk> <unk> <unk> aerosmith en janvier m...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""adaptation...","[{""keyword"": ""eaux"", ""timestamp"": 169354246806...",1,383,Cette allégation nécessite une vérification po...,les initiatives positives de jean z une nova s...,Dans un flux d'informations sur les initiative...,N/A,facts,medium,pro-écologie,les initiatives positives de jean z une nova s...,narrative
1,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,L'allégation semble être fondée sur des donnée...,La facture s'allège de manière non négligeable...,Dans une discussion sur une nouvelle innovatio...,N/A,facts,low,pro-écologie,La facture s'allège de manière non négligeable...,narrative
2,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,Cet énoncé reflète un changement de perception...,Les propriétaires et les campeurs ne considère...,Lors de l'explication de l'innovation mise en ...,N/A,narrative,low,pro-écologie,Les propriétaires et les campeurs ne considère...,narrative
3,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,Cette allégation semble provoquer un besoin de...,Il y a encore peu tenter ensuite à s'installer...,"Lors de l'entretien, il est fait mention de l'...",N/A,facts,medium,pro-écologie,Il y a encore peu tenter ensuite à s'installer...,narrative
4,0eb5805fa23e0819f817ea10fe1fccd19e61e40a1239cc...,2023-09-01 06:50:00,la très grande majorité d'entre eux ne connais...,europe1,True,Information - Magazine,Bonjour,"[""biodiversite_concepts_generaux"", ""changement...","[{""keyword"": ""eaux"", ""timestamp"": 169354380309...",3,593,Cette affirmation sur la croissance du nombre ...,Le nombre de campings équipés a doublé depuis ...,Dans le cadre de l'innovation dans la gestion ...,N/A,facts,medium,pro-écologie,Le nombre de campings équipés a doublé depuis ...,narrative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75308,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,Cette affirmation peut refléter une tendance o...,Les ventes de modèles électriques ont diminué ...,Il a été noté que les ventes de voitures élect...,Il peut y avoir des variations dans les ventes...,facts,medium,anti-écologie,"En juillet, les ventes de modèles électriques ...",narrative
75309,c5fb91f75955000b6bc581b89a9ba6cc0efda7b47b0b24...,2024-08-31 20:18:00,modèles rechargeables séduisent les citadins i...,tf1,False,Information - Journal,JT 20h + météo,"[""attenuation_climatique_solutions"", ""changeme...","[{""keyword"": ""borne de recharge"", ""timestamp"":...",2,503,Il est bien documenté que les incitations gouv...,La réduction des aides gouvernementales en Fra...,Le commentaire impliquait que la baisse des su...,N/A,facts,very low,pro-écologie,Une des c

In [93]:
df_claims

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,cards_label_pred
67,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
87,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
251,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative,0_accepted
259,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative,5_science_is_unreliable
313,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative,5_science_is_unreliable
...,...,...,...,...,...,...,...,...,...,...
73775,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative,0_accepted
74166,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,narrative,3_its_not_bad
74219,"Cette affirmation semble contestable, car de n...",Le risque de chutes de séracs n'est pas partic...,"Le chercheur indique que, pour l'instant, le r...",N/A,consensus,high,anti-écologie,"Pour l'instant, le risque de chutes de séracs ...",narrative,1_its_not_happening
74850,Cette information est manifestement fausse et ...,Une astuce sur les réseaux sociaux prétend que...,Un segment aborde une nouvelle tendance sur le...,N/A,narrative,high,anti-écologie,Cette astuce elle est fausse et voilà pourquoi.,narrative,0_accepted


In [ ]:
df

In [91]:
df_result.iloc[0].claim

"les initiatives positives de jean z une nova sion pour économiser l'eau au camping"

In [96]:
df[df.id == 'f17cb36a314156417616ef1ebb9232bbf203a1ecdc983c']

,id,start,text,channel_name,channel_is_radio,channel_program_type,channel_program,themes,keywords,num_keywords,num_tokens,claims


In [95]:
df.shape

(39131, 12)

In [103]:
df.iloc[76].claims

[{'analysis': "Cette allégation fait référence à des critiques concernant le Green Deal et son impact sur l'agriculture. Cependant, elle simplifie un sujet complexe en liant directement la souffrance des agriculteurs à des normes environnementales sans fournir de preuves à l'appui. Il est nécessaire de vérifier l'ampleur et la nature exacte de ces normes, ainsi que leur effet sur l'agriculture.",
  'claim': 'Les agriculteurs souffrent à cause des normes liées au Green Deal et au réchauffement climatique.',
  'context': 'Philippe de Villiers évoque la situation précaire des agriculteurs, soulignant que les normes environnementales impactent leur activité en raison du Green Deal et du changement climatique.',
  'contradiction': 'N/A',
  'disinformation_category': 'narrative',
  'disinformation_score': 'medium',
  'pro_anti': 'anti-écologie',
  'quote': "au nom du green deal du réchauffement climatique on leur inflige d'énormes discriminatoire par rapport aux paysans du monde entier",
  '

In [22]:
df_claims

,analysis,claim,context,contradiction,disinformation_category,disinformation_score,pro_anti,quote,speaker,cards_label_pred
67,Cette allégation semble associée à une interpr...,le giec est transhumaniste,Dans un passage en parlant des personnages du ...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
87,Cette allégation semble utiliser le terme 'tra...,"La mention du GIEC comme un ""transhumaniste"".",Dans un extrait de programme où différents per...,N/A,consensus,high,anti-écologie,il y a par exemple le giec transhumaniste qui ...,narrative,5_science_is_unreliable
251,Cette allégation semble hautement exagérée et ...,Il y a eu beaucoup de morts tués par des pharm...,Lors d'une discussion sur le rôle des pharmaci...,N/A,narrative,high,anti-écologie,j'ai l'impression qu'il y a eu beaucoup de mor...,narrative,0_accepted
259,"Cette allégation frôle la théorie du complot, ...",Le réchauffement climatique est causé par une ...,Lors d'une discussion sur les enjeux environne...,N/A,consensus,high,anti-écologie,Le réchauffement climatique est causé par une ...,narrative,5_science_is_unreliable
313,Si le changement climatique est parfois utilis...,Le changement climatique est traité comme une ...,Le commentaire sur une polémique entre personn...,N/A,consensus,high,anti-écologie,...c'était hier soir dans cette à vous sur fra...,narrative,5_science_is_unreliable
...,...,...,...,...,...,...,...,...,...,...
73775,Cette allégation est sérieuse et nécessite une...,L'incendie serait d'origine volontaire.,Le journaliste fait état des enquêtes en cours...,N/A,consensus,high,anti-écologie,d'après les autorités le feu serait d'origine ...,narrative,0_accepted
74166,Ce claim fait l'impasse sur le fait que la gén...,La climatisation industrialisée a été une répo...,"Il est mentionné que la climatisation, qui est...",N/A,consensus,high,anti-écologie,le progrès sur le réchauffement climatique la ...,narrative,3_its_not_bad
74219,"Cette affirmation semble contestable, car de n...",Le risque de chutes de séracs n'est pas partic...,"Le chercheur indique que, pour l'instant, le r...",N/A,consensus,high,anti-écologie,"Pour l'instant, le risque de chutes de séracs ...",narrative,1_its_not_happening
74850,Cette information est manifestement fausse et ...,Une astuce sur les réseaux sociaux prétend que...,Un segment aborde une nouvelle tendance sur le...,N/A,narrative,high,anti-écologie,Cette astuce elle est fausse et voilà pourquoi.,narrative,0_accepted


### test 

In [12]:
labels = [json.loads(item)["cards_label_pred"] for item in classification] 
labels

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [10]:
df_claims["cards_label_pred"] = labels

/var/folders/qs/h69xc93902zdpt61kf3m_p1r0000gn/T/ipykernel_4370/16650547.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample["cards_label_pred"] = labels
